# Mitochondria Displacement Plots
Script that goes through analyzed data and makes plots of mitochondria displacements
### File location parameters

In [1]:
analyzed_data_dir = '/Users/adam/Documents/SenseOfTouchResearch/SSN_ImageAnalysis/AnalyzedData/'

### Import packages

In [2]:
import yaml
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.spatial import distance
import warnings
import glob

### Functions for creating dataframes

In [3]:
def get_experiment_df(analyzed_data_dir):
    experiment_days = glob.iglob(analyzed_data_dir + '*')
    all_metadata = []
    for day in experiment_days:
        try:
            with open(day + '/metadata.yaml') as metadata_file:
                all_metadata.append(yaml.safe_load(metadata_file))
        except FileNotFoundError:
            pass

    metadata_df = pd.DataFrame(all_metadata)
    
    return metadata_df


def make_trial_df(experiment_id, all_metadata):
    metadata = all_metadata.loc[all_metadata['Experiment_id'] == experiment_id]
    mito_locations_file = analyzed_data_dir + experiment_id + '/trackpyBatchResults.yaml'
    with open(mito_locations_file, 'r') as yamlfile:
        mito_locations = yaml.safe_load(yamlfile)

    linked_mitos = pd.DataFrame.from_dict(mito_locations, orient='index')
    mito_locations = linked_mitos.loc[:, ['frame', 'particle', 'x', 'y', 'z']]
    
    mito_locations['pressure'] = mito_locations.apply(
        lambda row: get_pressure_this_mito(row), axis=1)
    
    mito_locations['x_rest'] = mito_locations.apply(
        lambda row: get_rest_location(row['particle'], 'x'), axis=1)
    mito_locations['y_rest'] = mito_locations.apply(
        lambda row: get_rest_location(row['particle'], 'y'), axis=1)
    mito_locations['z_rest'] = mito_locations.apply(
        lambda row: get_rest_location(row['particle'], 'z'), axis=1)

    return mito_locations


def make_mito_pair_df(mito_locations_df):
    mito_pairs_dicts = []
    for frame in mito_locations.frame.unique():
        this_frame = mito_locations.loc[(mito_locations['frame'] == frame)]
        this_frame.sort_values(by=['y'], inplace=True)
        this_frame.reset_index(inplace=True)
        # make dataframe of adjacent mitos
        for this_particle in range(mito_locations.particle.nunique() - 1):
            x_1 = this_frame.iloc[this_particle]['x']
            x_2 = this_frame.iloc[this_particle + 1]['x']
            x_dist = abs(x_2 - x_1)

            y_1 = this_frame.iloc[this_particle]['y']
            y_2 = this_frame.iloc[this_particle + 1]['y']
            y_dist = abs(y_2 - y_1)

            z_1 = this_frame.iloc[this_particle]['z']
            z_2 = this_frame.iloc[this_particle + 1]['z']
            z_dist = abs(z_2 - z_1)

            total_dist = distance.euclidean([x_1, y_1, z_1], [x_2, y_2, z_2])

            particle_1 = int(this_frame.iloc[this_particle]['particle'])
            particle_2 = int(this_frame.iloc[this_particle + 1]['particle'])

            mito_pairs_dicts.append({'x_dist': x_dist, 'x_1': x_1, 'x_2': x_2,
                                     'y_dist': y_dist, 'y_1': y_1, 'y_2': y_2,
                                     'z_dist': z_dist, 'z_1': z_1, 'z_2': z_2,
                                     'particle_1': particle_1,
                                     'particle_2': particle_2,
                                     'total_dist': total_dist, 'frame': frame,
                                     'pair_id': this_particle})

    mito_pairs = pd.DataFrame(mito_pairs_dicts)
    mito_pairs['pressure'] = mito_pairs.apply(lambda row: get_pressure_this_mito(row), axis=1)
    
    mito_pairs['rest_dist'] = mito_pairs.apply(
        lambda pair: get_rest_distance(pair['pair_id'], 'euclid'), axis=1)
    mito_pairs['x_rest_dist'] = mito_pairs.apply(
        lambda pair: get_rest_distance(pair['pair_id'], 'x'), axis=1)
    mito_pairs['y_rest_dist'] = mito_pairs.apply(
        lambda pair: get_rest_distance(pair['pair_id'], 'y'), axis=1)
    mito_pairs['z_rest_dist'] = mito_pairs.apply(
        lambda pair: get_rest_distance(pair['pair_id'], 'z'), axis=1)

    mito_pairs['strain'] = ((mito_pairs['total_dist'] - mito_pairs['rest_dist'])
                        / mito_pairs['rest_dist'])
    mito_pairs['x_strain'] = ((mito_pairs['x_dist'] - mito_pairs['x_rest_dist'])
                          / mito_pairs['x_rest_dist'])
    mito_pairs['y_strain'] = ((mito_pairs['y_dist'] - mito_pairs['y_rest_dist'])
                          / mito_pairs['y_rest_dist'])
    mito_pairs['z_strain'] = ((mito_pairs['z_dist'] - mito_pairs['z_rest_dist'])
                          / mito_pairs['z_rest_dist'])

    return mito_pairs

### Functions for adding information to existing dataframe

In [4]:
def get_pressure_this_mito(row):
    frame_num = int(row['frame'])
    pressure = all_metadata.loc[all_metadata['Experiment_id'] == 'SSN_120_001']['pressure_kPa'].tolist()
    pressure = pressure[0]
    pressure
    
    pressure = metadata['pressure_kPa'][frame_num]
    
    return pressure


def get_rest_location(particle, axis):

    x_rest = mito_locations.loc[(mito_locations['particle'] == particle) &
                                (mito_locations['pressure'] == 0)]['x'].mean()
    y_rest = mito_locations.loc[(mito_locations['particle'] == particle) &
                                (mito_locations['pressure'] == 0)]['y'].mean()
    z_rest = mito_locations.loc[(mito_locations['particle'] == particle) &
                                (mito_locations['pressure'] == 0)]['z'].mean()
    if axis == 'x':
        rest_location = x_rest
    elif axis == 'y':
        rest_location = y_rest
    elif axis == 'z':
        rest_location = z_rest
    else:
        raise ValueError("Invalid axes value")

    return rest_location


def get_rest_distance(pair, axis):
    euclidean_rest_dist = mito_pairs.loc[(mito_pairs['pair_id'] == pair) &
                                         (mito_pairs['pressure'] == 0)][
                                                      'total_dist'].mean()
    x_rest_dist = mito_pairs.loc[(mito_pairs['pair_id'] == pair) &
                                 (mito_pairs['pressure'] == 0)][
                                         'x_dist'].mean()
    y_rest_dist = mito_pairs.loc[(mito_pairs['pair_id'] == pair) &
                                 (mito_pairs['pressure'] == 0)][
                                         'y_dist'].mean()
    z_rest_dist = mito_pairs.loc[(mito_pairs['pair_id'] == pair) &
                                 (mito_pairs['pressure'] == 0)][
                                         'z_dist'].mean()
    if axis == 'euclid':
        rest_distance = euclidean_rest_dist
    elif axis == 'x':
        rest_distance = x_rest_dist
    elif axis == 'y':
        rest_distance = y_rest_dist
    elif axis == 'z':
        rest_distance = z_rest_dist
    else:
        raise ValueError("Invalid axes value")

    return rest_distance

### Function for plotting

In [5]:
# pressure = all_metadata.loc[all_metadata['Experiment_id'] == 'SSN_120_001']['pressure_kPa'].tolist()
# pressure = pressure[0]
# pressure

In [6]:
# get all the metadata
all_metadata = get_experiment_df(analyzed_data_dir)
# filter out trials we don't want
trials_with_strain_and_actuator = all_metadata.loc[(all_metadata['analysis_status'] == 'Strain calculated') & (all_metadata['actuator_center'].notnull())]
# make plots for those trials
# for index, row in metadata_df.iterrows():

# this_trial_mitos = make_trial_df('SSN_120_001', all_metadata)

In [10]:
experiment_id = 'SSN_168_001'
metadata = trials_with_strain_and_actuator.loc[all_metadata['Experiment_id'] == experiment_id]
mito_locations_file = analyzed_data_dir + experiment_id + '/trackpyBatchResults.yaml'
with open(mito_locations_file, 'r') as yamlfile:
    mito_locations = yaml.safe_load(yamlfile)
mito_locations = pd.DataFrame(mito_locations)
trials_with_strain_and_actuator


,Experiment_id,actuator_center,actuator_corners,actuator_thickness,analysis_status,bleach_time,cultivation_temp,device_ID,head_orientation,microscope,...,slice_height_pix,slice_width_pix,stack_height,timestamp,total_images,trial_rating,user,vulva_orientation,worm_life_stage,worm_strain
18,SSN_120_001,"[179.0, 591.5]","[180.0, 475.0, 178.0, 708.0]",26.034112,Strain calculated,2018-12-17 09:30:00,25,Y4P7-5,Headfirst,Spinning Disk Confocal in CSIF Shriram,...,1200,600,69,2018-12-19 14:52:43,759,,anekimke@stanford.edu,West,YA,NM3573
19,SSN_121_001,"[210.5, 443.0]","[210.0, 335.0, 211.0, 551.0]",20.990730,Strain calculated,2018-12-17 09:30:00,25,Y4P7-5,Headfirst,Spinning Disk Confocal in CSIF Shriram,...,1200,600,69,2018-12-19 15:03:24,759,,anekimke@stanford.edu,West,YA,NM3573
61,SSN_162_001,"[222.5, 581.5]","[228.0, 491.0, 217.0, 672.0]",49.884090,Strain calculated,2019-02-10 10:00:00,20,Y4P7-3,Tailfirst,Spinning Disk Confocal in CSIF Shriram,...,1070,600,56,2019-02-13 14:08:57,616,⭐️⭐️⭐️,anekimke@stanford.edu,East,YA,NM3573
62,SSN_163_001,"[383.5, 628.0]","[388.0, 524.0, 379.0, 732.0]",28.801532,Strain calculated,2019-02-10 10:00:00,20,Y4P7-3,Tailfirst,Spinning Disk Confocal in CSIF Shriram,...,1070,600,52,2019-02-13 14:26:40,572,⭐️⭐️⭐️,anekimke@stanford.edu,West,YA,NM3573
64,SSN_165_001,"[222.5, 585.0]","[225.0, 504.0, 220.0, 666.0]",27.089217,Strain calculated,2019-02-10 10:00:00,20,Y4P7-3,Tailfirst,Spinning Disk Confocal in CSIF Shriram,...,1070,600,60,2019-02-13 14:53:39,660,⭐️⭐️⭐️,anekimke@stanford.edu,East,YA,NM3573
65,SSN_166_001,"[370.5, 609.5]","[374.0, 507.0, 367.0, 712.0]",22.964126,Strain calculated,2019-02-10 10:00:00,20,Y4P7-3,Headfirst,Spinning Disk Confocal in CSIF Shriram,...,1070,600,57,2019-02-13 15:02:15,627,⭐️⭐️⭐️⭐️,anekimke@stanford.edu,West,YA,NM3573
66,SSN_167_001,"[213.5, 596.5]","[219.0, 511.0, 208.0, 682.0]",26.308598,Strain calculated,2019-02-10 10:00:00,20,Y4P7-3,Tailfirst,Spinning Disk Confocal in CSIF Shriram,...,1070,600,53,2019-02-13 15:17:12,583,⭐️⭐️,anekimke@stanford.edu,East,YA,NM3573
67,SSN_168_001,"[407.5, 594.0]","[410.0, 497.0, 405.0, 691.0]",21.999336,Strain calculated,2019-02-10 10:00:00,20,Y4P7-3,Tailfirst,Spinning Disk Confocal in CSIF Shriram,...,1070,600,60,2019-02-13 15:32:47,660,⭐️⭐️⭐️⭐️,anekimke@stanford.edu,West,YA,NM3573


## Testing functions to make sure analysis works properly

In [8]:
import unittest

class TestDataframeBuilders(unittest.TestCase):
        
    def setUp(self):
        self.analyzed_data_dir = '/Users/adam/Documents/SenseOfTouchResearch/SSN_ImageAnalysis/AnalyzedData/'
        self.all_metadata = get_experiment_df(self.analyzed_data_dir)
        self.test_cases = 'SSN_120_001'

    def test_make_trial_df(self):
        for exp_id in self.test_cases:
            this_trial_mitos = make_trial_df(exp_id, self.all_metadata)
            




unittest.main(argv=[''], verbosity=2, exit=False)

SyntaxError: invalid syntax (<ipython-input-8-ac44484d0cde>, line 7)